##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Networks for Image Classification



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>



```
# This is formatted as code
```

In this exercise we will use [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network to classify images.
2. Train and evaluate the precision of the network.
3. Build a convolutional network.
4. Learn the concepts of data augmentation and batch processing.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

We'll start by installing TensorFlow, Keras and other libraries required:




In [ ]:
%load_ext tensorboard
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorboard
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
#from datetime import datetime
import datetime, os
import numpy as np
import random

We load and prepare the database [MNIST](http://yann.lecun.com/exdb/mnist/). We convert the images from integer to floating point:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

The database consists of 28x28 pixel images representing handwritten digits. 
Let's visualize some of the images and their labels, to see how they look.

In [ ]:
plt.figure(figsize=(10,10))
random_inds = np.random.choice(60000,36)
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(x_train[image_ind]), cmap=plt.cm.binary)
    plt.xlabel(y_train[image_ind])

Let's see one of the digits in more detail



In [ ]:
# just a little function for pretty printing a matrix
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

# now print!       

matprint(x_train[image_ind])

## 1.2 Neural network for classification of handwritten digits
First we will build a simple neural network consisting of two fully connected layers and apply this to the digit classification task. Our network will eventually generate a probability distribution over the 10-digit classes (0-9). This first architecture is shown below:
![alt text](https://drive.google.com/uc?id=1Vv85zm3n6ypGDlzXA2tlpbi9f9BQZRSf)


Using the 'tf.keras.Sequential' function we can create a network by adding layers to the model. The model.summary function allows us to see the details of the created model .


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Once created, we compile the model, chossing the cost-function and the optimizer that we will use to train it.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

We train and evaluate the model:

In [ ]:
# Definition of the callback for TensorBoard.
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#model.fit(x_train, y_train, epochs=5)
#Versión con salida a Tensorboard
model.fit(x_train, y_train, epochs=5,callbacks=[tensorboard_callback])
model.evaluate(x_test,  y_test, verbose=2)

The image classifier has already been trained up to an accuracy of 98% in this dataset.



In [ ]:
%tensorboard --logdir logs  # Llamada a Tensorboard

# Checking the result
It is always useful to look at some results, to get an idea of when the network fails. With the following code, we show some examples where the network has succeeded and others where it failed.

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
#predicted_classes = model.predict_classes(x_test)
predicted_classes = np.argmax(model.predict(x_test), axis=-1)
# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]

incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.tight_layout()
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))
    
plt.tight_layout()

## 1.2 Convolutional Neural Network
We are now going to classify the data using a CNN.
To do that, we must reformat the input images, as the network expects a x*y*z image as input (z being the number of channels)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
print("Training matrix shape", x_train.shape)
print("Testing matrix shape", x_test.shape)

Now, using Sequential, we crear a convolutional network, we compile it and we train it

In [ ]:
cnn_model = tf.keras.Sequential([

        tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3), input_shape=(28, 28,1), activation=tf.nn.relu,name='ConvLayer01'),       

        tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2,padding='valid'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2,padding='valid'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
    
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_model.fit(x_train, y_train, epochs=1)

cnn_model.evaluate(x_test,  y_test, verbose=2)

## Data Augmentation
Let's increase the database using Data Augmentation. This can be useful for increasing the number of samples, or for preventing overfitting, by applying random variations to the data

In [ ]:
# Keras includes a function to perform data augmentation

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

We can now introduce the augmented data in batches


In [ ]:
train_generator = gen.flow(x_train, y_train, batch_size=128)
test_generator = test_gen.flow(x_test, y_test, batch_size=128)

And we adjust the model using the batch loader. This reduces a lot the memory needed for training

In [ ]:
cnn_model.fit_generator(train_generator, steps_per_epoch=60000//128, epochs=5, verbose=1, 
                    validation_data=test_generator, validation_steps=10000//128)